In [48]:
# dot product of two 1d arrays
def dot(a,b):
    sum = 0
    for i in range(len(a)):
        sum += a[i] * b[i];
    return sum
def norm(a):
    sum = 0
    for i in range(len(a)):
        sum += a[i] * a[i]
    return sum ** 0.5
def cossim(a,b):
    return dot(a,b) / (norm(a) * norm(b))

In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

import nltk
from nltk.tokenize import word_tokenize

In [50]:
df = pd.read_json('reviews_Musical_Instruments_5.json', lines=True)

In [51]:
#for test
#df = df[['asin','reviewerName','reviewText']][:50]
df = df[['asin','reviewerName','reviewText']]

In [52]:
# vectorize each review with 2000 features
tfidf = TfidfVectorizer(sublinear_tf=True,analyzer='word',max_features=2000,tokenizer= word_tokenize)

vectorizedReviews = tfidf.fit_transform(df['reviewText']).toarray()

/Users/YangLiu/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [67]:
df['maxId'] = pd.Series()
df['similarity'] = pd.Series()

In [68]:
# get the cosine similarity of every pair of the reviews.

for i in range(len(df['reviewText'])):
    maxsim = 0
    maxid = -1
    for j in range(len(df['reviewText'])):
        if i == j:
            continue
        tempSim = cossim(vectorizedReviews[i],vectorizedReviews[j])
        if (tempSim > maxsim):
            maxsim = tempSim
            maxid = j
    df['maxId'].iloc[i] = maxid
    df['similarity'].iloc[i] = maxsim

/Users/YangLiu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [116]:
# add a 0.06 weight to the similarity for the same product bought between two maxsimilar users
for i in range(len(df['reviewText'])):
    if df['asin'].iloc[int(df['maxId'].iloc[i])] == df['asin'].iloc[i]:
        df['similarity'].iloc[i] += 0.06

/Users/YangLiu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [117]:
# suggest friends

df['friend'] = pd.Series()

for i in range(len(df['reviewText'])):
    df['friend'].iloc[i] = df['reviewerName'].iloc[int(df['maxId'].iloc[i])]

/Users/YangLiu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [118]:
print("Top potential friendships are: " )
top_friendships_df = df.sort_values('similarity',ascending=False)[['reviewerName','friend']]
top_friendships_df

Top potential friendships are: 


,reviewerName,friend
38,"R. Wristen ""The Party Doc VJ and Karaoke""",Mark King
36,Mark King,"R. Wristen ""The Party Doc VJ and Karaoke"""
49,"C. Hill ""CFH""",NaN
29,Dan Edman,Mark King
11,Christopher C,"Wilhelmina Zeitgeist ""coolartsybabe"""
15,"Wilhelmina Zeitgeist ""coolartsybabe""",Christopher C
14,Straydogger,Christopher C
6,Brian,Kurt Robair
9,Kurt Robair,Brian
1,Jake,"cassandra tu ""Yeah, well, that's just like, u..."


In [119]:
print(top_friendships_df['reviewerName'].iloc[0], " said: ",  df['reviewText'].iloc[top_friendships_df.index[0]])
print("She or he bought item id: ", df['asin'].iloc[top_friendships_df.index[0]],'\n')
friendindex = int(df['maxId'].iloc[top_friendships_df.index[0]])
print(top_friendships_df['friend'].iloc[0], " said: ", df['reviewText'].iloc[friendindex])
print("She or he bought item id: ", df['asin'].iloc[friendindex],  '\n')
print('Congratulations to be the most potential friends!')



R. Wristen "The Party Doc VJ and Karaoke"  said:  I've used a lot of cables and I always come back to HOSA, they are indeed some of the best audio cables in their price range on the market.
She or he bought item id:  B000068NVI 

Mark King  said:  These are not the greatest but they're cheap and they get to you fast when you need them. I've only had one fail and I've bought many of them to use in our broadcast studio.Amazon is the first place I go when I need cables for audio, video, computers or concert lighting.Good music to all!
She or he bought item id:  B000068NVI 

Congratulations to be the most potential friends!
